In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sxs
import scri
from pathlib import Path
import urllib.request
import json
import sys
import subprocess

plt.style.use('seaborn-talk')
plt.rcParams["figure.figsize"] = (12, 10)

bbh_ext_folder = Path("/panfs/ds09/sxs/himanshu/cache/sxs_ext")


## Code to download files for abd from bbh_ext catalog

In [ ]:
"""
input: https://zenodo.org/record/4734836/files/Lev5/r3Psi2OverM_BondiCce_R0513_CoM.h5?download=1
output: links of all files with same radius and Lev that are required to make a abd
"""
def get_info_from_file_link(link:str):
  zonedo_link = f'https://zenodo.org/record/{link.split("/")[4]}'
  Lev = link.split("/")[-2]
  Radius = link.split("/")[-1].split("_")[2][1:]

  data_list = ["rhOverM","rMPsi4","r2Psi3","r3Psi2OverM","r4Psi1OverM2","r5Psi0OverM3"]
  download_links = [f"{zonedo_link}/files/{Lev}/{data_file}_BondiCce_R{Radius}_CoM.h5" for data_file in data_list]
  download_links_json = [f"{zonedo_link}/files/{Lev}/{data_file}_BondiCce_R{Radius}_CoM.json" for data_file in data_list]

  link_dict = {
    "metadata": f"{zonedo_link}/files/{Lev}/metadata.json"
  }
  for i,data_name in enumerate(data_list):
    link_dict[data_name] = download_links[i]
    link_dict[data_name+"_json"] = download_links_json[i]

  return link_dict


"""
input: https://zenodo.org/record/4734836/files/Lev5/r3Psi2OverM_BondiCce_R0513_CoM.h5?download=1
output: download files with same radius and Lev that are required to make a abd.
Returns the download folder path
"""
def create_folder_and_save_data(link:str):
    link_dict = get_info_from_file_link(link)
    # save metadata file and use it to get the folder name
    url = link_dict['metadata']
    response = urllib.request.urlopen(url)
    data = response.read().decode('utf-8')     # a `bytes` object

    temp_json_metadata_file = bbh_ext_folder/"del.json"
    with temp_json_metadata_file.open("w") as f:
        f.write(data)

    with temp_json_metadata_file.open('r') as f:
        metadata = json.load(f)

    # bbh_ext_folder is a global variable
    save_folder_name:Path = bbh_ext_folder/metadata['simulation_name']

    # Make sure the folder does not exists
    if save_folder_name.exists():
        sys.exit(f"{save_folder_name} already exists.")

    save_folder_name.mkdir(parents=True,exist_ok=False)

    for key in link_dict.keys():
        download_command = f"cd {save_folder_name} && wget {link_dict[key]}"
        status = subprocess.run(download_command,
                                capture_output=True,
                                shell=True,
                                text=True)
        if status.returncode == 0:  
            print(f'Succesfully downloaded {link_dict[key].split("/")[-1]} in {save_folder_name}' )
        else:
            sys.exit(
                f"Failed to download {link_dict[key]}"
            )

    return save_folder_name

# Given a folder finds all .h5 files in it
def path_dict_in_folder(data_path:Path):
    path_dict = {}
    for file_path in list(data_path.glob("*.h5")):
        path_dict[str(file_path).split("/")[-1].split("_")[0]] = file_path

    return path_dict

  

In [ ]:
link_dict = get_info_from_file_link(link)

## 

In [ ]:
catalog = sxs.load("catalog")
dataframe = catalog.table
dataframe


In [ ]:
waveform = sxs.load("SXS:BBH:0123/Lev/rhOverM", extrapolation_order=2)
plt.plot(waveform.t, waveform.data.real)


## Download and check data

In [ ]:
link = "https://zenodo.org/record/4734836/files/Lev3/r3Psi2OverM_BondiCce_R0513_CoM.h5?download=1"
link_dict = create_folder_and_save_data(link)
link_dict


### load all the folders with data in a dict

In [ ]:
folder_file_dict = {}
for folder in bbh_ext_folder.iterdir():
    if not folder.is_dir():
        continue
    else:
        # print(folder)
        folder_file_dict[str(folder.stem)] = {}
        for lev_folder in folder.iterdir():
            print(lev_folder)
            folder_file_dict[str(folder.stem)][str(
                lev_folder.stem)] = path_dict_in_folder(lev_folder)

folder_file_dict


In [ ]:
folder_file_dict['bbh_q1_nospin']['Lev3']['rhOverM'].exists()


In [ ]:
data_dict = folder_file_dict['bbh_q1_nospin']['Lev3']
abd = scri.SpEC.file_io.create_abd_from_h5(h=str(data_dict['rhOverM']),
                                           Psi4=str(
                                               data_dict['rMPsi4']),
                                           Psi3=str(
                                               data_dict['r2Psi3']),
                                           Psi2=str(
                                               data_dict['r3Psi2OverM']),
                                           Psi1=str(
                                               data_dict['r4Psi1OverM2']),
                                           Psi0=str(
                                               data_dict['r5Psi0OverM3']),
                                           file_format='sxs')


In [ ]:
scri.rpxmb.load(str(data_dict['rhOverM']))


In [ ]:
sxs.rpdmb.load(str(data_dict['rhOverM']))


In [ ]:
abd = scri.SpEC.file_io.create_abd_from_h5(h=str(cce_path/currrent_lev/f'extracted_data/rhOverM_BondiCce_R{currrent_lev[-4:]}.h5'),
                                           Psi4=str(
                                               cce_path/currrent_lev/f'extracted_data/rMPsi4_BondiCce_R{currrent_lev[-4:]}.h5'),
                                           Psi3=str(
                                               cce_path/currrent_lev/f'extracted_data/r2Psi3_BondiCce_R{currrent_lev[-4:]}.h5'),
                                           Psi2=str(
                                               cce_path/currrent_lev/f'extracted_data/r3Psi2OverM_BondiCce_R{currrent_lev[-4:]}.h5'),
                                           Psi1=str(
                                               cce_path/currrent_lev/f'extracted_data/r4Psi1OverM2_BondiCce_R{currrent_lev[-4:]}.h5'),
                                           Psi0=str(
                                               cce_path/currrent_lev/f'extracted_data/r5Psi0OverM3_BondiCce_R{currrent_lev[-4:]}.h5'),
                                           file_format='SXS')


In [ ]:
zonedo_link = "https://zenodo.org/record/4734836"
Lev = "Lev3"
Radius = "0292"

data_list = ["rhOverM", "rMPsi4", "r2Psi3",
             "r3Psi2OverM", "r4Psi1OverM2", "r5Psi0OverM3"]
download_links = [
    f"{zonedo_link}/files/{Lev}/{data_file}_BondiCce_R{Radius}_CoM.h5" for data_file in data_list]
download_links


In [ ]:
link = "https://zenodo.org/record/4734836/files/Lev3/r3Psi2OverM_BondiCce_R0513_CoM.h5?download=1"
link_dict = get_info_from_file_link(link)
link_dict
